Here we'll put the bulk of the work.

In [1]:
import pandas as pd
import numpy as np

v_df = pd.read_csv(
    'datasets/visits_log_us.csv',
    parse_dates=['start_time', 'end_time'],
    dtype=
        {
            'device': 'category',
            'source_id': 'category'
        }
    )
o_df = pd.read_csv(
    'datasets/orders_log_us.csv',
    parse_dates=['purchase_time']
)
c_df = pd.read_csv(
    'datasets/costs_us.csv',
    parse_dates=['date'],
    dtype=
        {
            'source_id': 'category'
        }
)

# for checking on the data, but only one line needs to be commented out or un-commented out
check_data = False
# check_data = True
if check_data:
    v_df.info()
    print(v_df.head())
    print()
    o_df.info()
    print(o_df.head())
    print()
    c_df.info()
    print(c_df.head())

In [2]:
v_df['session_year']  = v_df['start_time'].dt.isocalendar().year
v_df['session_month'] = v_df['start_time'].dt.month
v_df['session_week']  = v_df['start_time'].dt.isocalendar().week
v_df['session_date'] = v_df['start_time'].dt.date

print(v_df.sample(7))

         device            end_time source_id          start_time  \
155216  desktop 2018-02-05 17:55:00         2 2018-02-05 17:44:00   
198856    touch 2018-02-24 12:41:00         4 2018-02-24 12:16:00   
269628    touch 2018-01-02 14:10:00         5 2018-01-02 13:43:00   
174000  desktop 2017-10-03 17:08:00         4 2017-10-03 17:07:00   
170586  desktop 2018-04-09 10:08:00         1 2018-04-09 09:48:00   
103452  desktop 2017-11-17 17:36:00        10 2017-11-17 17:35:00   
359029  desktop 2017-10-07 12:47:19         9 2017-10-07 12:47:00   

                         uid  session_year  session_month  session_week  \
155216  11579244902408609464          2018              2             6   
198856   1617420242868300797          2018              2             8   
269628   7345909147862244936          2018              1             1   
174000   8849966431667305406          2017             10            40   
170586   4101460083485262047          2018              4            15 

In [3]:
dau_total = (
    v_df.groupby('session_date')
    .agg({'uid': 'nunique'})
    .mean()
)

wau_total = (
    v_df.groupby(['session_year', 'session_week'])
    .agg({'uid': 'nunique'})
    .mean()
)

mau_total = (
    v_df.groupby(['session_year', 'session_month'])
    .agg({'uid': 'nunique'})
    .mean()
)

sticky_wau = (dau_total / wau_total) * 100
sticky_mau = (dau_total / mau_total) * 100

print(f'dau = {int(dau_total)} wau = {int(wau_total)} mau = {int(mau_total)}')
print(f'sticky_wau = {float(sticky_wau):.2f}% sticky_mau = {float(sticky_mau):.2f}%')

dau = 907 wau = 5716 mau = 23228
sticky_wau = 15.88% sticky_mau = 3.91%


In [4]:
v_df['session_duration_mins'] = (
    v_df['end_time'] - v_df['start_time']
).dt.seconds / 60

v_df[['uid', 'start_time', 'session_duration_mins']].sample(10)

,uid,start_time,session_duration_mins
124027,728936768711112455,2017-10-20 23:02:00,47.0
223030,8131943237643870202,2017-08-07 00:02:00,2.0
29443,13060475859206605623,2017-09-05 18:40:00,2.0
202942,18027222598828835800,2017-08-30 10:07:00,19.0
241080,7396922114699819335,2018-02-27 07:25:00,6.0
58115,18313200315307807852,2018-03-01 11:41:00,1.0
99127,513372882715687297,2017-11-14 10:46:00,6.0
122976,3124140781886341267,2017-12-29 01:06:00,10.0
29121,15462019830439323784,2017-08-02 12:09:00,5.0
326299,6192820683259771119,2017-10-14 22:15:00,0.0


In [5]:
v_by_day = v_df.groupby('session_date').agg({
    'start_time': 'count'
})
v_by_day = v_by_day.rename(
    columns={
        'start_time': 'sessions'
    }
)
v_by_day.sample(10)

,sessions
session_date,
2018-03-19,1303
2017-09-28,1184
2017-08-24,530
2017-09-08,881
2017-11-01,1339
2018-05-28,1156
2018-03-25,1703
2018-05-07,774
2017-12-19,1280


In [6]:
first_session_date = v_df.groupby(['uid'])['session_date'].min()
first_session_date.name = 'first_session_date'

v_df = v_df.join(first_session_date, on='uid')

In [7]:
first_session_week = v_df.groupby(['uid'])['session_week'].min()
first_session_week.name = 'first_session_week'

v_df = v_df.join(first_session_week, on='uid')

In [8]:
first_session_month = v_df.groupby(['uid'])['session_month'].min()
first_session_month.name = 'first_session_month'

v_df = v_df.join(first_session_month, on='uid')

In [9]:
v_df['cohort_lifetime_months'] = ((
        v_df['session_date']
            -
        v_df['first_session_date']
    ) / np.timedelta64(1, 'M')
).astype(int)
v_df['cohort_lifetime_weeks'] = ((
        v_df['session_date']
            -
        v_df['first_session_date']
    ) / np.timedelta64(1, 'W')
).astype(int)
v_df['cohort_lifetime_days'] = ((
        v_df['session_date']
            -
        v_df['first_session_date']
    ) / np.timedelta64(1, 'D')
).astype(int)

v_df[['uid', 'cohort_lifetime_days', 'cohort_lifetime_weeks', 'cohort_lifetime_months']].sample(10)

,uid,cohort_lifetime_days,cohort_lifetime_weeks,cohort_lifetime_months
242722,4394506233053867282,262,37,8
270192,10340246027996293586,0,0,0
337056,11682391024948207287,0,0,0
30196,14140126450209050662,0,0,0
51030,18304746974697862628,0,0,0
56283,6152954986509545665,2,0,0
49689,11897672804968866743,0,0,0
270618,18241499569920743518,0,0,0
205548,6028141585119740972,0,0,0
47115,8482256677254277956,0,0,0


In [10]:
cohorts_monthly = (
    v_df.groupby(['first_session_month', 'cohort_lifetime_months'])
    .agg({'uid': 'nunique'})
    .reset_index()
)

initial_users_count = cohorts_monthly[cohorts_monthly['cohort_lifetime_months'] == 0][ ['first_session_month', 'uid'] ]
initial_users_count = initial_users_count.rename(
    columns={'uid': 'monthly_cohort_users'}
) 

cohorts_monthly = cohorts_monthly.merge(initial_users_count, on='first_session_month')

cohorts_monthly['retention'] = cohorts_monthly['uid'] / cohorts_monthly['monthly_cohort_users']

cohorts_monthly[['first_session_month', 'monthly_cohort_users', 'retention']].sample(10)

,first_session_month,monthly_cohort_users,retention
45,4,17468,0.012938
5,1,28716,0.048997
42,4,17468,0.018892
2,1,28716,0.098865
30,3,23510,0.019651
72,7,11450,0.014847
9,1,28716,0.012746
58,5,16834,0.011821
88,12,22073,1.000000
1,1,28716,0.118053


In [14]:
# make and print table
retention_pivot = cohorts_monthly.pivot_table(
    index='first_session_month',
    columns='cohort_lifetime_months',
    values='retention',
    aggfunc='sum',
)

retention_pivot.fillna('')

cohort_lifetime_months,0,1,2,3,4,5,6,7,8,9,10,11
first_session_month,,,,,,,,,,,,
1,1.0,0.118053,0.098865,0.084343,0.062021,0.048997,0.043634,0.032908,0.017203,0.012746,0.009472,0.006233
2,1.0,0.061945,0.065675,0.051102,0.037913,0.026839,0.022186,0.020841,0.015342,0.004999,0.003538,0.00173
3,1.0,0.041684,0.035006,0.034368,0.03373,0.02769,0.019651,0.014037,0.015355,0.01157,0.003063,0.001148
4,1.0,0.028681,0.006526,0.013453,0.026219,0.024216,0.018892,0.012079,0.008759,0.012938,0.00727,0.001775
5,1.0,0.008554,0.00499,0.004574,0.011287,0.019425,0.018177,0.015386,0.009445,0.007366,0.011821,0.009029
6,1.0,0.034397,0.032187,0.038288,0.034928,0.033336,0.018216,,,,,
7,1.0,0.035284,0.03607,0.033188,0.033362,0.014847,0.000087,,,,,
8,1.0,0.043823,0.041923,0.034097,0.013639,,,,,,,
9,1.0,0.051454,0.043475,0.012711,,,,,,,,


In [ ]:
first_purchase_date = o_df.groupby(['uid'])['purchase_time'].min()
first_purchase_date.name = 'first_purchase_date'

user_conversion = (
    first_session_date.to_frame()
    .join(first_purchase_date, on='uid')
    .dropna()
    # .fillna('never')
)
user_conversion.reset_index(inplace=True)
user_conversion['conversion_days'] = ((
        user_conversion['first_purchase_date']
            -
        pd.to_datetime(user_conversion['first_session_date'])
    ) / np.timedelta64(1, 'D')
).astype(int)

user_conversion.sample(10)

In [ ]:
user_conversion['conversion_days'].value_counts()

In [ ]:
v_df[[
    'uid', 
    'cohort_lifetime_months', 
    'cohort_lifetime_weeks', 
    'cohort_lifetime_days'
]].sample(10)

In [ ]:
o_df = o_df.merge(user_conversion[['uid', 'first_purchase_date']], on='uid', how='left')

o_df.head()

In [ ]:
o_df['user_age'] = ((
        o_df['purchase_time']
            -
        o_df['first_purchase_date']
    ) / np.timedelta64(1, 'D')
)

o_df.sample(10)

In [ ]:
# Group o_df by uid and calculate the total number of purchases per uid
purchase_count = o_df.groupby('uid')['purchase_time'].count()

# Calculate the duration in days, weeks, and months for each uid
duration_days = (o_df.groupby('uid')['purchase_time'].max() - o_df.groupby('uid')['purchase_time'].min()).dt.days
duration_weeks = duration_days / 7
duration_months = duration_days / 30

# Calculate the average number of purchases per day, week, and month
avg_purchases_per_day = np.where(duration_days == 0, 0, purchase_count / duration_days)
avg_purchases_per_week = np.where(duration_weeks == 0, 0, purchase_count / duration_weeks)
avg_purchases_per_month = np.where(duration_months == 0, 0, purchase_count / duration_months)

# Combine the results into a new DataFrame
average_purchases_df = pd.DataFrame({
    'uid': purchase_count.index,
    'avg_purchases_per_day': avg_purchases_per_day,
    'avg_purchases_per_week': avg_purchases_per_week,
    'avg_purchases_per_month': avg_purchases_per_month
})

# Display the average purchases per day, week, and month
average_purchases_df.sample(20)


In [ ]:
# Group o_df by uid and calculate the total profit and number of purchases per uid
purchase_stats = o_df.groupby('uid').agg({'profit': 'sum', 'purchase_time': 'count'})

# Calculate the average purchase size per user
average_purchase_size_per_user = purchase_stats['profit'] / purchase_stats['purchase_time']

# Add the average purchase size per user to o_df
o_df = o_df.merge(average_purchase_size_per_user.rename('average_purchase_size_per_user'), on='uid')

# Display the average purchase size per user
average_purchase_size_per_user_df = o_df[['uid', 'average_purchase_size_per_user']].drop_duplicates()
average_purchase_size_per_user_df.sample(10)


In [ ]:
total_ltv_data = o_df.groupby('uid')['profit'].sum().reset_index()

total_ltv_data = total_ltv_data.sort_values('profit', ascending=False)

total_ltv_data.head(20)


In [ ]:
purchase_stats = o_df.groupby('uid').agg({'profit': 'sum', 'purchase_time': 'count'})

purchase_stats.sample(20)

average_purchase_size = purchase_stats['profit'] / purchase_stats['purchase_time']

print(average_purchase_size)


In [ ]:
engagement_duration_months = (o_df.groupby('uid')['purchase_time'].max() - o_df.groupby('uid')['first_purchase_date'].min()) / np.timedelta64(1, 'M')
ltv_data['ltv'] = average_purchase_size * engagement_duration_months
ltv_data = ltv_data.sort_values('ltv', ascending=False)
print(engagement_duration_months)

ltv_data['ltv'].head(20)